# Calibration of the school model based on empirical data

In [4]:
import numpy as np
import pandas as pd
import numpy as np
from os.path import join
import json

# custom functions to run the calibration simulations
import calibration_functions as cf

# for progress bars
from ipywidgets import IntProgress
from IPython.display import display

## Empirical observations

In [5]:
empirical_data_src = '../../data/calibration/empirical_observations'

# distribution of outbreak sizes by school type
outbreak_sizes = pd.read_csv(\
            join(empirical_data_src, 'empirical_outbreak_sizes.csv'))
# ratio of infections in the student and teacher groups
group_distributions = pd.read_csv(\
            join(empirical_data_src, 'empirical_group_distributions.csv'))
# ratio of index cases in the student and teacher groups
agent_index_ratios = pd.read_csv(\
            join(empirical_data_src, 'empirical_index_case_ratios.csv'))
agent_index_ratios.index = agent_index_ratios['school_type']
# ratio of symptomatic cases in the student and teacher groups
symptomatic_case_ratios = pd.read_csv(\
            join(empirical_data_src, 'empirical_symptomatic_case_ratios.csv'))

## Simulation settings

In [6]:
# List of prevention measures that were in place in schools in the weeks 36-45
# of the year 2020 in Austrian schools. This list was compiled from information
# collected in interviews with teachers of different school types. NOTE: so far
# there are no recorded differences between school types.
with open('params/calibration_measures.json', 'r') as fp:
    prevention_measures = json.load(fp)
# simulation parameters, specifically the
# - base transmission risk (calibrated from household transmissions)
# - subclinical transmission modifier (literature value)
# - exposure duration, time until symtpoms and infection duration (lit. values)
# - age symptom discount (fit to empirical observations)
with open('params/calibration_simulation_parameters.json', 'r') as fp:
    simulation_params = json.load(fp)
# characteristics (# classes, # students / class, # teachers) of the "average" 
# school, depending on school type. These characteristics were determined in 
# interviews with Austrian teachers and from statistics about Austrian schools 
# (year 2017/18, page 10: https://www.bmbwf.gv.at/Themen/schule/schulsystem/gd.html)
# NOTE: "students" indicates the number of students per class
with open('params/calibration_school_characteristics.json', 'r') as fp:
    school_characteristics = json.load(fp)

## Random sampling

### Create calibration parameter grid

In [7]:
## grid of parameters that need to be calibrated
# the contact weight is the modifier by which the base transmission risk (for
# household transmissions) is multiplied for contacts of type "intermediate" 
# and of type "far"
intermediate_contact_weights = np.arange(0, 1, 0.05)
far_contact_weights = np.arange(0, 1, 0.05)

# the age_transmission_discount sets the slope of the age-dependence of the 
# transmission risk. Transmission risk for adults (age 18+) is always base 
# transmission risk. For every year an agent is younger than 18 years, the
# transmission risk is reduced
age_transmission_discounts = np.arange(-0.1, 0, 0.02)

# list of all possible parameter combinations from the grid
params = [(i, j, k) for i in intermediate_contact_weights \
                    for j in far_contact_weights\
                    for k in age_transmission_discounts if i > j]

# number of points in the parameter grid that will be randomly sampled
N_samples = 10
# randomly drawn list of parameter combination indices of size N_samples. For 
# every parameter combination an ensemble of simulations will be run and 
# evaluated.
if N_samples == 'all':
    samples = range(len(params))
else:
    samples = np.random.choice(range(len(params)), N_samples, replace=False)

### Run the sampling

In [8]:
# paths for data I/O
contact_network_src = '../../data/calibration/contact_networks'
dst = '../../data/calibration/simulation_results'

school_types = ['primary', 'primary_dc', 'lower_secondary',
                'lower_secondary_dc', 'upper_secondary', 'secondary']

# number of runs per ensemble
# Note: this is set to 1 for testing purposes. To get properly converged 
# statistics, this has to be >= 500. Running such a high number of simulations
# will take a long time if run on just a single core. Therefore I strongly
# recommend to run the below code on several cores at once. It is easy to
# parallelize as each ensemble can be run on a different core and there are
# no interdependencies between ensembles. Results can be collected afterwards
# and evaulated together.
N_runs = 1

# progress bar
f = IntProgress(min=0, max=len(school_types) * len(samples)) 
display(f)
c = 0

results = pd.DataFrame()
for school_type in school_types:
    for sample_index in samples:
        # get the randomly sampled parameter combination
        intermediate_contact_weight, far_contact_weight,\
            age_transmission_discount = params[sample_index]
        
        # run the ensemble with the given parameter combination and school type
        ensemble_results = cf.run_ensemble(N_runs, school_type,
                  intermediate_contact_weight, far_contact_weight, 
                  age_transmission_discount, prevention_measures,
                  school_characteristics, agent_index_ratios,
                  simulation_params, contact_network_src)

        # calculate the difference between the ensemble characteristics
        # (outbreak distribution, ratio of student to teacher cases)
        row = cf.evaluate_ensemble(ensemble_results, school_type,
                  intermediate_contact_weight, far_contact_weight,
                  age_transmission_discount, outbreak_sizes,
                  group_distributions)

        results = results.append(row, ignore_index=True)
        
        f.value = c # update the progress bar
        c += 1 # update progress bar

results.reset_index()
index_cols = ['school_type', 'intermediate_contact_weight',
              'far_contact_weight', 'age_transmission_discount']
other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

results.to_csv(join(dst, 'calibration_results_random_sampling.csv'), index=False)

IntProgress(value=0, max=60)

### Analyze the random sampling

In [21]:
# load the collected ensemble statistics
dst = '../../data/calibration/simulation_results'
results = pd.read_csv(join(dst, 'calibration_results_random_sampling.csv'))
results.head(3)

,age_transmission_discount,chi2_distance_distro,chi2_distance_size,chi2_distance_total,far_contact_weight,intermediate_contact_weight,school_type,sum_of_squares_distro,sum_of_squares_size,sum_of_squares_total
0,-0.02,0.090392,0.080200,0.170592,0.15,0.45,primary,0.133439,0.101209,0.234648
1,-0.02,0.061329,0.007015,0.068345,0.70,0.95,primary,0.090537,0.007860,0.098396
2,-0.04,0.078039,0.046071,0.124110,0.10,0.85,primary,0.115204,0.057563,0.172767


In [22]:
# note: these are the number of clusters per school type from the slightly older
# data version (November 2020). For primary and lower secondary school types,
# the counts are split evenly between schools with and without daycare. 
counts = pd.DataFrame({'type':['primary', 'primary_dc', 
                               'lower_secondary', 'lower_secondary_dc',
                               'upper_secondary', 'secondary'],
                      'count':[33.5, 33.5, 90, 90, 116, 70]})

counts.index = counts['type']
counts = counts.drop(columns=['type'])

# The cluster counts are used to weigh the respective school type in the 
# calibration process.
counts['weight'] = counts['count'] / counts['count'].sum()
results['chi2_distance_total_weighted'] = results['chi2_distance_total']
weights = []
for i, row in results.iterrows():
    st = row['school_type']
    weight = counts.loc[st, 'weight']
    error = row['chi2_distance_total']
    results.loc[i, 'chi2_distance_total_weighted'] = error * weight

In [23]:
# since we calibrate the intermediate_contact_weight, far_contact_weight and 
# age_transmission_discount over all school types together, we have to find
# the parameter combination that minimizes the sum of errors over all school
# types. 
agg_results = results.groupby(['intermediate_contact_weight',
                               'far_contact_weight',
                               'age_transmission_discount']).sum()

opt = agg_results.loc[agg_results['chi2_distance_total_weighted'].idxmin()].name
opt_intermediate_contact_weight = opt[0]
opt_far_contact_weight = opt[1]
opt_age_transmission_discount = opt[2]

print('optimal random sampling parameter combination:')
print('\t intermediate contact weight: {:1.3f}'.format(opt_intermediate_contact_weight))
print('\t far contact weight: {:1.3f}'.format(opt_far_contact_weight))
print('\t age transmission discount: {:1.3f}'.format(opt_age_transmission_discount))

optimal random sampling parameter combination:
	 intermediate contact weight: 0.850
	 far contact weight: 0.750
	 age transmission discount: -0.020


## Grid search

### Create calibration parameter grid

In [51]:
## grid of parameters that need to be calibrated
# the contact weight is the modifier by which the base transmission risk (for
# household transmissions) is multiplied for contacts of type "intermediate" 
# and of type "far". Parameter values are chosen around the optimum from the
# previous random sampling search
intermediate_contact_weights = [0.825, 0.875]
far_contact_weights = [0.725, 0.75]

# the age_transmission_discount sets the slope of the age-dependence of the 
# transmission risk. Transmission risk for adults (age 18+) is always base 
# transmission risk. For every year an agent is younger than 18 years, the
# transmission risk is reduced. Parameter values are chosen around the optimum 
# from the previous random sampling search
age_transmission_discounts = [-0.015,-0.0175,-0.0225,-0.025,-0.0275]

# list of all possible parameter combinations from the grid
params = [(i, j, k) for i in intermediate_contact_weights \
                    for j in far_contact_weights\
                    for k in age_transmission_discounts if i > j]

### Run the sampling

In [52]:
# paths for data I/O
contact_network_src = '../../data/calibration/contact_networks'
dst = '../../data/calibration/simulation_results'

school_types = ['primary', 'primary_dc', 'lower_secondary',
                'lower_secondary_dc', 'upper_secondary', 'secondary']

# number of runs per ensemble
# Note: this is set to 1 for testing purposes. To get properly converged 
# statistics, this has to be >= 500. Running such a high number of simulations
# will take a long time if run on just a single core. Therefore I strongly
# recommend to run the below code on several cores at once. It is easy to
# parallelize as each ensemble can be run on a different core and there are
# no interdependencies between ensembles. Results can be collected afterwards
# and evaulated together.
N_runs = 1

# progress bar
f = IntProgress(min=0, max=len(school_types) * len(params)) 
display(f)
c = 0

results = pd.DataFrame()
for school_type in school_types:
    for p in params:
        # get the randomly sampled parameter combination
        intermediate_contact_weight, far_contact_weight,\
            age_transmission_discount = p
        
        # run the ensemble with the given parameter combination and school type
        ensemble_results = cf.run_ensemble(N_runs, school_type,
                  intermediate_contact_weight, far_contact_weight, 
                  age_transmission_discount, prevention_measures,
                  school_characteristics, agent_index_ratios,
                  simulation_params, contact_network_src)

        # calculate the difference between the ensemble characteristics
        # (outbreak distribution, ratio of student to teacher cases)
        row = cf.evaluate_ensemble(ensemble_results, school_type,
                  intermediate_contact_weight, far_contact_weight,
                  age_transmission_discount, outbreak_sizes,
                  group_distributions)

        results = results.append(row, ignore_index=True)
        
        f.value = c
        c += 1 # update progress bar

results.reset_index()
index_cols = ['school_type', 'intermediate_contact_weight',
              'far_contact_weight', 'age_transmission_discount']
other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

# combine the results from the random sampling with the grid results
#random_results = pd.read_csv(join(dst, 'calibration_results_random_sampling.csv'))
#results = pd.concat([random_results, results])
#results.to_csv(join(dst, 'calibration_results_grid_search.csv'), index=False)

IntProgress(value=0, max=120)

### Analyze the grid search

In [46]:
# load the collected ensemble statistics
dst = '../../data/calibration/simulation_results'
grid_results = pd.read_csv(join(dst, 'calibration_results_grid_search.csv'))
grid_results.head(3)

,age_transmission_discount,chi2_distance_distro,chi2_distance_size,chi2_distance_total,far_contact_weight,intermediate_contact_weight,school_type,sum_of_squares_distro,sum_of_squares_size,sum_of_squares_total
0,-0.02,0.090392,0.080200,0.170592,0.15,0.45,primary,0.133439,0.101209,0.234648
1,-0.02,0.061329,0.007015,0.068345,0.70,0.95,primary,0.090537,0.007860,0.098396
2,-0.04,0.078039,0.046071,0.124110,0.10,0.85,primary,0.115204,0.057563,0.172767


In [47]:
counts = pd.DataFrame({'type':['primary', 'primary_dc', 
                               'lower_secondary', 'lower_secondary_dc',
                               'upper_secondary', 'secondary'],
                      'count':[33.5, 33.5, 90, 90, 116, 70]})
counts.index = counts['type']
counts = counts.drop(columns=['type'])
counts['weight'] = counts['count'] / counts['count'].sum()
grid_results['chi2_distance_total_weighted'] = grid_results['chi2_distance_total']
weights = []
for i, row in grid_results.iterrows():
    st = row['school_type']
    weight = counts.loc[st, 'weight']
    error = row['chi2_distance_total']
    grid_results.loc[i, 'chi2_distance_total_weighted'] = error * weight

In [49]:
agg_results = grid_results.groupby(['age_transmission_discount', 'intermediate_contact_weight',
                 'far_contact_weight']).sum()
opt = agg_results.loc[agg_results['chi2_distance_total_weighted'].idxmin()].name
opt_intermediate_contact_weight = opt[0]
opt_far_contact_weight = opt[1]
opt_age_transmission_discount = opt[2]

print('optimal grid search parameter combination:')
print('\t intermediate contact weight: {:1.3f}'.format(opt_intermediate_contact_weight))
print('\t far contact weight: {:1.3f}'.format(opt_far_contact_weight))
print('\t age transmission discount: {:1.3f}'.format(opt_age_transmission_discount))

optimal grid search parameter combination:
	 intermediate contact weight: -0.020
	 far contact weight: 0.850
	 age transmission discount: 0.750
